In [18]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import os

In [19]:
def orb_bf_match_and_find(query_img, train_img, query_img_filename, train_img_filename, min_matches): 
    # Scale train_img to be the same size as query_img
    train_img = cv.resize(train_img, (query_img.shape[1], query_img.shape[0]))
    # Create an ORB object
    orb = cv.ORB_create()
    
    features1, des1 = orb.detectAndCompute(query_img, None)
    features2, des2 = orb.detectAndCompute(train_img, None)

    # Create Brute-Force matcher object
    bf = cv.BFMatcher(cv.NORM_HAMMING)
    matches = bf.knnMatch(des1, des2, k = 2)
    
    # Nearest neighbour ratio test to find good matches
    good = []    
    good_without_lists = []    
    matches = [match for match in matches if len(match) == 2] 
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
            good_without_lists.append(m)
         
    if len(good) >= min_matches:
        # Draw a polygon around the recognized object
        src_pts = np.float32([features1[m.queryIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        dst_pts = np.float32([features2[m.trainIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        
        # Get the transformation matrix
        M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
               
        # Find the perspective transformation to get the corresponding points
        h, w = query_img.shape[:2]
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv.perspectiveTransform(pts, M)
        
        train_img = cv.polylines(train_img, [np.int32(dst)], True, (0, 255, 0), 2, cv.LINE_AA)
    else:
        print('Not enough good matches are found - {}/{}'.format(len(good), min_matches))
    
    print('Total matches found: {}'.format(len(good)))        
    result_img = cv.drawMatchesKnn(query_img, features1, train_img, features2, good, None, flags = 4)
    cv.putText(result_img, 'Total matches found: {}'.format(len(good)), (200, 200), cv.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 2, cv.LINE_AA)
    cv.imwrite('output/orb/bf/{}_and_{}.jpg'.format(query_img_filename, train_img_filename), result_img)

def orb_bf_match_with_images_folder(template, template_filename, min_matches):
    for filename in os.listdir('output/orb/bf'):
        if filename.startswith(template_filename):
            os.remove('output/orb/bf/' + filename)
    for filename in os.listdir('images'):
        img = cv.imread('images/' + filename)
        filename_no_ext = os.path.splitext(filename)[0]
        orb_bf_match_and_find(template, img, template_filename, filename_no_ext, min_matches)

In [20]:
# Running matching with remote control template
min_matches = 10
template = cv.imread('templates/remote-1.jpg')
orb_bf_match_with_images_folder(template, 'remote-1', min_matches)
template = cv.imread('templates/remote-2.jpg')
orb_bf_match_with_images_folder(template, 'remote-2', min_matches)

Not enough good matches are found - 8/10
Total matches found: 8
Not enough good matches are found - 9/10
Total matches found: 9
Not enough good matches are found - 3/10
Total matches found: 3
Not enough good matches are found - 8/10
Total matches found: 8
Not enough good matches are found - 8/10
Total matches found: 8
Not enough good matches are found - 8/10
Total matches found: 8
Total matches found: 21
Total matches found: 16
Total matches found: 12
Total matches found: 29
Total matches found: 49
Total matches found: 17
Not enough good matches are found - 5/10
Total matches found: 5
Not enough good matches are found - 9/10
Total matches found: 9
Not enough good matches are found - 3/10
Total matches found: 3
Not enough good matches are found - 9/10
Total matches found: 9
Not enough good matches are found - 5/10
Total matches found: 5
Not enough good matches are found - 2/10
Total matches found: 2
Total matches found: 14
Total matches found: 12
Total matches found: 15
Total matches fo

In [21]:
# Running matching with earphone template
min_matches = 25
template = cv.imread('templates/earphone-1.jpg')
orb_bf_match_with_images_folder(template, 'earphone-1', min_matches)
template = cv.imread('templates/earphone-2.jpg')
orb_bf_match_with_images_folder(template, 'earphone-2', min_matches)

Not enough good matches are found - 18/25
Total matches found: 18
Not enough good matches are found - 19/25
Total matches found: 19
Total matches found: 31
Not enough good matches are found - 23/25
Total matches found: 23
Not enough good matches are found - 17/25
Total matches found: 17
Not enough good matches are found - 13/25
Total matches found: 13
Not enough good matches are found - 11/25
Total matches found: 11
Total matches found: 40
Not enough good matches are found - 20/25
Total matches found: 20
Not enough good matches are found - 11/25
Total matches found: 11
Not enough good matches are found - 12/25
Total matches found: 12
Total matches found: 27
Not enough good matches are found - 23/25
Total matches found: 23
Not enough good matches are found - 11/25
Total matches found: 11
Not enough good matches are found - 18/25
Total matches found: 18
Not enough good matches are found - 20/25
Total matches found: 20
Not enough good matches are found - 14/25
Total matches found: 14
Not 

In [22]:
# Running matching with elephant template
min_matches = 8
template = cv.imread('templates/elephant-1.jpg')
orb_bf_match_with_images_folder(template, 'elephant-1', min_matches)
template = cv.imread('templates/elephant-2.jpg')
orb_bf_match_with_images_folder(template, 'elephant-2', min_matches)

Not enough good matches are found - 7/8
Total matches found: 7
Not enough good matches are found - 2/8
Total matches found: 2
Total matches found: 9
Total matches found: 11
Not enough good matches are found - 4/8
Total matches found: 4
Total matches found: 8
Total matches found: 8
Not enough good matches are found - 7/8
Total matches found: 7
Not enough good matches are found - 7/8
Total matches found: 7
Not enough good matches are found - 6/8
Total matches found: 6
Total matches found: 8
Total matches found: 15
Not enough good matches are found - 7/8
Total matches found: 7
Total matches found: 8
Not enough good matches are found - 5/8
Total matches found: 5
Not enough good matches are found - 6/8
Total matches found: 6
Not enough good matches are found - 4/8
Total matches found: 4
Not enough good matches are found - 2/8
Total matches found: 2
Not enough good matches are found - 6/8
Total matches found: 6
Not enough good matches are found - 4/8
Total matches found: 4
Not enough good ma

In [23]:
def orb_flann_match_and_find(query_img, train_img, query_img_filename, train_img_filename, min_matches): 
    # Scale train_img to be the same size as query_img
    train_img = cv.resize(train_img, (query_img.shape[1], query_img.shape[0]))
    # Create an ORB object
    orb = cv.ORB_create()
    
    features1, des1 = orb.detectAndCompute(query_img, None)
    features2, des2 = orb.detectAndCompute(train_img, None)

    # Create a FLANN matcher object
    index_params = dict(algorithm = 6, table_number = 6, key_size = 12, multi_probe_level = 1)
    search_params = dict()

    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k = 2)
    
    # Nearest neighbour ratio test to find good matches
    good = []    
    good_without_lists = []    
    matches = [match for match in matches if len(match) == 2] 
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
            good_without_lists.append(m)
         
    if len(good) >= min_matches:
        # Draw a polygon around the recognized object
        src_pts = np.float32([features1[m.queryIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        dst_pts = np.float32([features2[m.trainIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        
        # Get the transformation matrix
        M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
               
        # Find the perspective transformation to get the corresponding points
        h, w = query_img.shape[:2]
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv.perspectiveTransform(pts, M)
        train_img = cv.polylines(train_img, [np.int32(dst)], True, (0, 255, 0), 2, cv.LINE_AA)
    else:
        print('Not enough good matches are found - {}/{}'.format(len(good), min_matches))
    
    print('Total matches found: {}'.format(len(good)))        
    result_img = cv.drawMatchesKnn(query_img, features1, train_img, features2, good, None, flags = 4)

    cv.putText(result_img, 'Total matches found: {}'.format(len(good)), (200, 200), cv.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 2, cv.LINE_AA)
    cv.imwrite('output/orb/flann/{}_and_{}.jpg'.format(query_img_filename, train_img_filename), result_img)

def orb_flann_match_with_images_folder(template, template_filename, min_matches):
    for filename in os.listdir('output/orb/flann'):
        if filename.startswith(template_filename):
            os.remove('output/orb/flann/' + filename)
    for filename in os.listdir('images'):
        img = cv.imread('images/' + filename)
        filename_no_ext = os.path.splitext(filename)[0]
        orb_flann_match_and_find(template, img, template_filename, filename_no_ext, min_matches)

In [24]:
# Running matching with remote control template
min_matches = 10
template = cv.imread('templates/remote-1.jpg')
orb_flann_match_with_images_folder(template, 'remote-1', min_matches)
template = cv.imread('templates/remote-2.jpg')
orb_flann_match_with_images_folder(template, 'remote-2', min_matches)


Total matches found: 21
Total matches found: 38
Total matches found: 15
Total matches found: 28
Total matches found: 18
Total matches found: 19
Total matches found: 34
Total matches found: 22
Total matches found: 20
Total matches found: 44
Total matches found: 69
Total matches found: 27
Total matches found: 18
Total matches found: 38
Not enough good matches are found - 8/10
Total matches found: 8
Total matches found: 28
Total matches found: 11
Total matches found: 16
Total matches found: 24
Total matches found: 24
Total matches found: 24
Total matches found: 35
Total matches found: 53
Total matches found: 31


In [28]:
def sift_bf_match_and_find(query_img, train_img, query_img_filename, train_img_filename, min_matches): 
    # Scale train_img to be the same size as query_img
    train_img = cv.resize(train_img, (query_img.shape[1], query_img.shape[0]))
    # Create an SIFT object
    sift = cv.SIFT_create()
    
    features1, des1 = sift.detectAndCompute(query_img, None)
    features2, des2 = sift.detectAndCompute(train_img, None)

    # Create a BFMatcher object
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1, des2, k = 2)
    
    # Nearest neighbour ratio test to find good matches
    good = []    
    good_without_lists = []    
    matches = [match for match in matches if len(match) == 2] 
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
            good_without_lists.append(m)
         
    if len(good) >= min_matches:
        # Draw a polygon around the recognized object
        src_pts = np.float32([features1[m.queryIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        dst_pts = np.float32([features2[m.trainIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        
        # Get the transformation matrix
        M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
               
        # Find the perspective transformation to get the corresponding points
        h, w = query_img.shape[:2]
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv.perspectiveTransform(pts, M)
        train_img = cv.polylines(train_img, [np.int32(dst)], True, (0, 255, 0), 2, cv.LINE_AA)
    else:
        print('Not enough good matches are found - {}/{}'.format(len(good), min_matches))
    
    print('Total matches found: {}'.format(len(good)))

    result_img = cv.drawMatchesKnn(query_img, features1, train_img, features2, good, None, flags = 4)
    cv.putText(result_img, 'Total matches found: {}'.format(len(good)), (200, 200), cv.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 2, cv.LINE_AA)
    cv.imwrite('output/sift/bf/{}_and_{}.jpg'.format(query_img_filename, train_img_filename), result_img)

def sift_bf_match_with_images_folder(template, template_filename, min_matches):
    for filename in os.listdir('output/sift/bf'):
        if filename.startswith(template_filename):
            os.remove('output/sift/bf/' + filename)
    for filename in os.listdir('images'):
        img = cv.imread('images/' + filename)
        filename_no_ext = os.path.splitext(filename)[0]
        sift_bf_match_and_find(template, img, template_filename, filename_no_ext, min_matches)
    

In [29]:
#Running matching with earphone template
min_matches = 10
template = cv.imread('templates/earphone-1.jpg')
sift_bf_match_with_images_folder(template, 'earphone-1', min_matches)
template = cv.imread('templates/earphone-2.jpg')
sift_bf_match_with_images_folder(template, 'earphone-2', min_matches)


Total matches found: 238
Total matches found: 209
Total matches found: 454
Total matches found: 266
Total matches found: 283
Total matches found: 462
Total matches found: 236
Total matches found: 204
Total matches found: 226
Total matches found: 222
Total matches found: 258
Total matches found: 183
Total matches found: 1523
Total matches found: 1318
Total matches found: 2248
Total matches found: 1953
Total matches found: 1992
Total matches found: 2399
Total matches found: 1457


In [ ]:
def sift_flann_match_and_find(query_img, train_img, query_img_filename, train_img_filename, min_matches): 
    # Scale train_img to be the same size as query_img
    train_img = cv.resize(train_img, (query_img.shape[1], query_img.shape[0]))
    # Create an SIFT object
    sift = cv.SIFT_create()
    
    features1, des1 = sift.detectAndCompute(query_img, None)
    features2, des2 = sift.detectAndCompute(train_img, None)

    # Create a FLANN matcher object
    index_params = dict(algorithm = 0, trees = 5)
    search_params = dict(checks = 50)
    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k = 2)
    
    # Nearest neighbour ratio test to find good matches
    good = []    
    good_without_lists = []    
    matches = [match for match in matches if len(match) == 2] 
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
            good_without_lists.append(m)
         
    if len(good) >= min_matches:
        # Draw a polygon around the recognized object
        src_pts = np.float32([features1[m.queryIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        dst_pts = np.float32([features2[m.trainIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        
        # Get the transformation matrix
        M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
               
        # Find the perspective transformation to get the corresponding points
        h, w = query_img.shape[:2]
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv.perspectiveTransform(pts, M)
        train_img = cv.polylines(train_img, [np.int32(dst)], True, (0, 255, 0), 2, cv.LINE_AA)
    else:
        print('Not enough good matches are found - {}/{}'.format(len(good), min_matches))

    print('Total matches found: {}'.format(len(good)))

    result_img = cv.drawMatchesKnn(query_img, features1, train_img, features2, good, None, flags = 4)
    cv.putText(result_img, 'Total matches found: {}'.format(len(good)), (200, 200), cv.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 2, cv.LINE_AA)
    cv.imwrite('output/sift/flann/{}_and_{}.jpg'.format(query_img_filename, train_img_filename), result_img)
    
def sift_flann_match_with_images_folder(template, template_filename, min_matches):
    for filename in os.listdir('output/sift/flann'):
        if filename.startswith(template_filename):
            os.remove('output/sift/flann/' + filename)
    for filename in os.listdir('images'):
        img = cv.imread('images/' + filename)
        filename_no_ext = os.path.splitext(filename)[0]
        sift_flann_match_and_find(template, img, template_filename, filename_no_ext, min_matches)

In [ ]:
# Running matching with elephant template
min_matches = 10
template = cv.imread('templates/elephant-1.jpg')
sift_flann_match_with_images_folder(template, 'elephant-1', min_matches)
template = cv.imread('templates/elephant-2.jpg')
sift_flann_match_with_images_folder(template, 'elephant-2', min_matches)